In [47]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from PIL import Image

In [48]:
shape_img = (28,28)
Epochs = 100
Lr_Rate = 1e-3
Batch_Size = 128

In [39]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [41]:
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=Batch_Size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=Batch_Size, shuffle=True)

In [44]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        #Encoder
        self.enc1 = nn.Linear(in_features=784, out_features=256) # Input image (28*28 = 784)
        self.enc2 = nn.Linear(in_features=256, out_features=128)
        self.enc3 = nn.Linear(in_features=128, out_features=64)
        self.enc4 = nn.Linear(in_features=64, out_features=32)
        self.enc5 = nn.Linear(in_features=32, out_features=16)

        #Decoder 
        self.dec1 = nn.Linear(in_features=16, out_features=32)
        self.dec2 = nn.Linear(in_features=32, out_features=64)
        self.dec3 = nn.Linear(in_features=64, out_features=128)
        self.dec4 = nn.Linear(in_features=128, out_features=256)
        self.dec5 = nn.Linear(in_features=256, out_features=784) # Output image (28*28 = 784)

    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        x = F.relu(self.enc5(x))

        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        x = F.relu(self.dec5(x))

        return x

In [52]:
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=Lr_Rate)
device=''

In [53]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

def make_dir():
    image_dir = 'MNIST_Out_Images'
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)

def save_decod_img(img, epoch):
    img = img.view(img.size(0), 1, 28, 28)
    save_image(img, './MNIST_Out_Images/Autoencoder_image{}.png'.format(epoch))


def training(model, train_loader, Epochs):
    train_loss = []
    for epoch in range(Epochs):
        running_loss = 0.0
        for data in train_loader:
            img, _ = data
            img = img.to(device)
            img = img.view(img.size(0), -1)
            optimizer.zero_grad()
            outputs = model(img)
            loss = criterion(outputs, img)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        loss = running_loss / len(train_loader)
        train_loss.append(loss)
        print('Epoch {} of {}, Train Loss: {:.3f}'.format(
            epoch+1, Epochs, loss))

        if epoch % 5 == 0:
            save_decod_img(outputs.cpu().data, epoch)

    return train_loss

def test_image_reconstruct(model, test_loader):
     for batch in test_loader:
        img, _ = batch
        img = img.to(device)
        img = img.view(img.size(0), -1)
        outputs = model(img)
        outputs = outputs.view(outputs.size(0), 1, 28, 28).cpu().data
        save_image(outputs, 'MNIST_reconstruction.png')
        break

In [54]:
device = get_device()
model.to(device)
make_dir()

In [55]:
train_loss = training(model, train_loader, Epochs)

Epoch 1 of 100, Train Loss: 0.922
Epoch 2 of 100, Train Loss: 0.917
Epoch 3 of 100, Train Loss: 0.912
Epoch 4 of 100, Train Loss: 0.908
Epoch 5 of 100, Train Loss: 0.904
Epoch 6 of 100, Train Loss: 0.902
Epoch 7 of 100, Train Loss: 0.900
Epoch 8 of 100, Train Loss: 0.898
Epoch 9 of 100, Train Loss: 0.897
Epoch 10 of 100, Train Loss: 0.896
Epoch 11 of 100, Train Loss: 0.896
Epoch 12 of 100, Train Loss: 0.895
Epoch 13 of 100, Train Loss: 0.894
Epoch 14 of 100, Train Loss: 0.893
Epoch 15 of 100, Train Loss: 0.892
Epoch 16 of 100, Train Loss: 0.892
Epoch 17 of 100, Train Loss: 0.892
Epoch 18 of 100, Train Loss: 0.891
Epoch 19 of 100, Train Loss: 0.891
Epoch 20 of 100, Train Loss: 0.890
Epoch 21 of 100, Train Loss: 0.890
Epoch 22 of 100, Train Loss: 0.890
Epoch 23 of 100, Train Loss: 0.890
Epoch 24 of 100, Train Loss: 0.889
Epoch 25 of 100, Train Loss: 0.889
Epoch 26 of 100, Train Loss: 0.889
Epoch 27 of 100, Train Loss: 0.888
Epoch 28 of 100, Train Loss: 0.888
Epoch 29 of 100, Train Loss: 

In [61]:
torch.save(model, 'model')
model = torch.load('model')
model.eval()

Autoencoder(
  (enc1): Linear(in_features=784, out_features=256, bias=True)
  (enc2): Linear(in_features=256, out_features=128, bias=True)
  (enc3): Linear(in_features=128, out_features=64, bias=True)
  (enc4): Linear(in_features=64, out_features=32, bias=True)
  (enc5): Linear(in_features=32, out_features=16, bias=True)
  (dec1): Linear(in_features=16, out_features=32, bias=True)
  (dec2): Linear(in_features=32, out_features=64, bias=True)
  (dec3): Linear(in_features=64, out_features=128, bias=True)
  (dec4): Linear(in_features=128, out_features=256, bias=True)
  (dec5): Linear(in_features=256, out_features=784, bias=True)
)